### Importing necessary libraries, packages and functions

In [ ]:
import csv
import pandas as pd
import string
from nltk import corpus
from nltk import word_tokenize
from nltk import pos_tag

### Load the tsv file

In [ ]:
title = ["Amazon EC2", "Amazon RDS", "Amazon S3", "Amazon CloudFront", "Amazon VPC", "Amazon SNS",
            "Amazon Elastic Beanstalk", "Amazon Lambda"]

fileloc = "C:\\Users\\Aruna\\Documents\\ACMS-IID\\Scraped Files\\Forums\\"

text = pd.read_csv(fileloc + title[0] + '.tsv', delimiter='\t', encoding='utf-8')
# text['description']

#### Step 1) Remove the question description and then the extra whitespace

In [ ]:
cleantext = text['description'].tolist()

for i in range(0, len(cleantext)):
    if cleantext[i][0] == 'R' and cleantext[i][1] == 'e' and cleantext[i][2] == ':':
        cleantext[i] = ' '.join(cleantext[i].splitlines()[1:])
    cleantext[i] = ' '.join(cleantext[i].split())

text['description'] = cleantext    
    
# text['description']

#### Step 2) Tokenize the text

In [ ]:
flag = 0
txt = []

for msg in text['description']:
    tokens = word_tokenize(msg)
    txt.append(tokens)
    # print(flag)
    flag = flag + 1

#### Step 3) Remove (some) common names and proper nouns from text

<i>Note: We are converting the names corpus to uppercase because otherwise checking is case sensitive </i>

In [ ]:
import nltk
nltk.download('names')
nltk.download('words')
# names = corpus.names.words()
words = corpus.words.words()

names = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER',
         'DECEMBER', 'JAN', 'FEB', 'MAR', 'APR', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC','MONDAY', 'TUESDAY',
         'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY', 'MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'
        ]

for name in corpus.names.words():
    names.append(name.upper())


In [ ]:
flag = 0
cleantext = []

for item in txt:
    mylist = []
    pos = pos_tag(item)
    for name, tag in pos:
        if tag == 'NNP' and name.upper() in names:
            # print(name)    
            flag = flag + 1
        else:
            mylist.append(name)
            
    cleantext.append(mylist)
            
flag

In [ ]:
m = len(cleantext)
for i in range(0, m):
    n = len(cleantext[i])
    for j in range(0, n):
        try:
            if len(cleantext[i][j]) < 2:
                del cleantext[i][j]
                n = n - 1
        except:
            j = j - 1

cleantext

In [ ]:
text['description'] = cleantext

#### Step 4) Remove stop words

In [ ]:
stop_words = corpus.stopwords.words('english')

new_stop_words = ['something', "'m", 'please', 'thank', 'thanks', 'hello', 'hi', 'hey', "'ve", 'regards', "ca", 'I', "'d",
                  "n\'t", "s3", "aws", "ec2", "regards", "'s", "help", "Edited", "sns", "cloudfront", "vpc", "rds", 
                  "beanstalk", "lambda", "Amazon", 'wo', "Elastic"]
stop_words.extend(new_stop_words)

cleantext = []


for i in range(0, len(stop_words)):
    stop_words[i] = stop_words[i].upper()

# print(stop_words)

m = len(text['description'])

for i in range(0, m):
    n = len(text['description'][i])
    
    sentence = []
    
    for j in range(0, n):
        if text['description'][i][j].upper() not in stop_words:
            sentence.append(text['description'][i][j])
    
    cleantext.append(sentence)

In [ ]:
text['description'] = cleantext

text['description']

#### Step 5) Remove all the 'words' that are only symbols and convert all words to lowercase

In [ ]:
cleantext = []

for msg in text['description']:
    sent = []
    for word in msg:
        if word.isalnum() == True:
            sent.append(word.lower())
    cleantext.append(sent)

cleantext

#### Step 6) Remove repeated words in the same sentence

In [ ]:
text['description'] = cleantext

cleantext = []

for msg in text['description']:
    sent = []
    for word in msg:
        if word not in sent:
            sent.append(word.lower())
    cleantext.append(sent)

cleantext


### Write to the data.csv file

In [ ]:
with open('C:\\Users\\Aruna\\Documents\\ACMS-IID\\Code\\Data Cleaning\\Data.csv', 'a', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['label', 'description'])
    for item in cleantext:
        if len(item) > 0:
            writer.writerow([title[0], item])